# Gathering Data

In [1]:
# Importing libs
import numpy as np
import pandas as pd

# all lightfm imports
from lightfm.data import Dataset
from lightfm import LightFM, cross_validation
from lightfm.evaluation import precision_at_k, auc_score # type: ignore

# imports re for text cleaning
import re
from datetime import datetime, timedelta

# we will ignore pandas warnings
import warnings
warnings.filterwarnings('ignore')

/opt/homebrew/anaconda3/envs/kaggle/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
# Read all datasets and store them in pandas df
base_path = './input/'
df_answer_scores = pd.read_csv(
    base_path + 'answer_scores.csv'
)

df_answers = pd.read_csv(
    base_path + 'answers.csv',
    parse_dates=['answers_date_added']
)

df_comments = pd.read_csv(
    base_path + 'comments.csv'
)

df_emails = pd.read_csv(
    base_path + 'emails.csv'
)

df_group_memberships = pd.read_csv(
    base_path + 'group_memberships.csv'
)

df_groups = pd.read_csv(
    base_path + 'groups.csv'
)

df_matches = pd.read_csv(
    base_path + 'matches.csv'
)

df_professionals = pd.read_csv(
    base_path + 'professionals.csv',
    parse_dates=['professionals_date_joined']
)

df_question_scores = pd.read_csv(
    base_path + 'question_scores.csv'
)

df_questions = pd.read_csv(
    base_path + 'questions.csv',
    parse_dates=['questions_date_added']
)

df_school_memberships = pd.read_csv(
    base_path + 'school_memberships.csv'
)

df_students = pd.read_csv(
    base_path + 'students.csv',
    parse_dates=['students_date_joined']
)

df_tag_questions = pd.read_csv(
    base_path + 'tag_questions.csv'
)

df_tag_users = pd.read_csv(
    base_path + 'tag_users.csv'
)

df_tags = pd.read_csv(
    base_path + 'tags.csv'
)

In [3]:
df_answer_scores.head()

,id,score
0,7b2bb0fc0d384e298cffa6afde9cf6ab,1
1,7640a6e5d5224c8681cc58de860858f4,5
2,3ce32e236fa9435183b2180fb213375c,2
3,fa30fe4c016043e382c441a7ef743bfb,0
4,71229eb293314c8a9e545057ecc32c93,2


In [4]:
df_answers.head()

,answers_id,answers_author_id,answers_question_id,answers_date_added,answers_body
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14+00:00,<p>Hi!</p>\n<p>You are asking a very interesti...
1,ada720538c014e9b8a6dceed09385ee3,2aa47af241bf42a4b874c453f0381bd4,eb80205482e4424cad8f16bc25aa2d9c,2018-05-01 14:19:08+00:00,<p>Hi. I joined the Army after I attended coll...
2,eaa66ef919bc408ab5296237440e323f,cbd8f30613a849bf918aed5c010340be,eb80205482e4424cad8f16bc25aa2d9c,2018-05-02 02:41:02+00:00,"<p>Dear Priyanka,</p><p>Greetings! I have answ..."
3,1a6b3749d391486c9e371fbd1e605014,7e72a630c303442ba92ff00e8ea451df,4ec31632938a40b98909416bdd0decff,2017-05-10 19:00:47+00:00,<p>I work for a global company who values high...
4,5229c514000446d582050f89ebd4e184,17802d94699140b0a0d2995f30c034c6,2f6a9a99d9b24e5baa50d40d0ba50a75,2017-10-13 22:07:33+00:00,I agree with Denise. Every single job I've had...


In [5]:
df_comments.head()

,comments_id,comments_author_id,comments_parent_content_id,comments_date_added,comments_body
0,f30250d3c2ca489db1afa9b95d481e08,9fc88a7c3323466dbb35798264c7d497,b476f9c6d9cd4c50a7bacdd90edd015a,2019-01-31 23:39:40 UTC+0000,"First, you speak to recruiters. They are train..."
1,ca9bfc4ba9464ea383a8b080301ad72c,de2415064b9b445c8717425ed70fd99a,ef4b6ae24d1f4c3b977731e8189c7fd7,2019-01-31 20:30:47 UTC+0000,Most large universities offer study abroad pro...
2,c354f6e33956499aa8b03798a60e9386,6ed20605002a42b0b8e3d6ac97c50c7f,ca7a9d7a95df471c816db82ee758f57d,2019-01-31 18:44:04 UTC+0000,"First, I want to put you at ease that the oppo..."
3,73a6223948714c5da6231937157e4cb7,d02f6d9faac24997a7003a59e5f34bd3,c7a88aa76f5f49b0830bfeb46ba17e4d,2019-01-31 17:53:28 UTC+0000,Your question submission was great! I just wan...
4,55a89a9061d44dd19569c45f90a22779,e78f75c543e84e1c94da1801d8560f65,c7a88aa76f5f49b0830bfeb46ba17e4d,2019-01-31 14:51:53 UTC+0000,Thank you. I'm new to this site. I'm sorry if ...


# Defining our necessary funcs

In [6]:
def generate_int_id(dataframe, id_col_name):
    """
    Generate unique integer id for users, questions and answers

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A. 
    id_col_name : String 
        New integer id's column name.
        
    Returns
    -------
    Dataframe
        Updated dataframe containing new id column 
    """
    new_dataframe=dataframe.assign(
        int_id_col_name=np.arange(len(dataframe))
        ).reset_index(drop=True)
    return new_dataframe.rename(columns={'int_id_col_name': id_col_name})

def create_features(dataframe, features_name, id_col_name):
    """
    Generate features that will be ready for feeding into lightfm

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe which contains features
    features_name : List
        List of feature columns name avaiable in dataframe
    id_col_name: String
        Column name which contains id of the question or
        answer that the features will map to.
        There are two possible values for this variable.
        1. questions_id_num
        2. professionals_id_num

    Returns
    -------
    Pandas Series
        A pandas series containing process features
        that are ready for feed into lightfm.
        The format of each value
        will be (user_id, ['feature_1', 'feature_2', 'feature_3'])
        Ex. -> (1, ['military', 'army', '5'])
    """

    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = list(zip(dataframe[id_col_name], features))
    return features

def generate_feature_list(dataframe, features_name):
    """
    Generate features list for mapping 

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A. 
    features_name: List
        List of feature columns name avaiable in dataframe. 
        
    Returns
    -------
    List of all features for mapping 
    """
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = features.apply(pd.Series).stack().reset_index(drop=True)
    return features

def calculate_auc_score(lightfm_model, interactions_matrix, 
                        question_features, professional_features): 
    """
    Measure the ROC AUC metric for a model. 
    A perfect score is 1.0.

    Parameters
    ----------
    lightfm_model: LightFM model 
        A fitted lightfm model 
    interactions_matrix: 
        A lightfm interactions matrix 
    question_features, professional_features: 
        Lightfm features 
        
    Returns
    -------
    String containing AUC score 
    """
    score = auc_score( 
        lightfm_model, interactions_matrix, 
        item_features=question_features, 
        user_features=professional_features, 
        num_threads=4).mean()
    return score

# Data Preprocessing and feature creation

In [7]:
# generating unique integer id for users and q&a
df_professionals = generate_int_id(df_professionals, 'professionals_id_num')
df_students = generate_int_id(df_students, 'students_id_num')
df_questions = generate_int_id(df_questions, 'questions_id_num')
df_answers = generate_int_id(df_answers, 'answers_id_num')

**Merging Datasets**

In [8]:
# merging dataset

# just dropna from tags 
df_tags = df_tags.dropna()
df_tags['tags_tag_name'] = df_tags['tags_tag_name'].str.replace('#', '')

# merge tag_questions with tags name
# then group all tags for each question into single rows
df_tags_question = df_tag_questions.merge(
    df_tags, how='inner',
    left_on='tag_questions_tag_id', right_on='tags_tag_id')
df_tags_question = df_tags_question.groupby(
    ['tag_questions_question_id'])['tags_tag_name'].apply(
        ','.join).reset_index()
df_tags_question = df_tags_question.rename(columns={'tags_tag_name': 'questions_tag_name'})

# merge tag_users with tags name 
# then group all tags for each user into single rows 
# after that rename the tag column name 
df_tags_pro = df_tag_users.merge(
    df_tags, how='inner',
    left_on='tag_users_tag_id', right_on='tags_tag_id')
df_tags_pro = df_tags_pro.groupby(
    ['tag_users_user_id'])['tags_tag_name'].apply(
        ','.join).reset_index()
df_tags_pro = df_tags_pro.rename(columns={'tags_tag_name': 'professionals_tag_name'})

# merge professionals and questions tags with main merge_dataset 
df_questions = df_questions.merge(
    df_tags_question, how='left',
    left_on='questions_id', right_on='tag_questions_question_id')
df_professionals = df_professionals.merge(
    df_tags_pro, how='left',
    left_on='professionals_id', right_on='tag_users_user_id')

# merge questions with scores 
df_questions = df_questions.merge(
    df_question_scores, how='left',
    left_on='questions_id', right_on='id')
# merge questions with students 
df_questions = df_questions.merge(
    df_students, how='left',
    left_on='questions_author_id', right_on='students_id')

# merge answers with questions 
# then merge professionals and questions score with that 
df_merge = df_answers.merge(
    df_questions, how='inner',
    left_on='answers_question_id', right_on='questions_id')
df_merge = df_merge.merge(
    df_professionals, how='inner',
    left_on='answers_author_id', right_on='professionals_id')
df_merge = df_merge.merge(
    df_question_scores, how='inner',
    left_on='questions_id', right_on='id')

**Generate some features**

In [9]:
# Generate some features for calculates weights
# that will use with interaction matrix 

df_merge['num_of_ans_by_professional'] = df_merge.groupby(['answers_author_id'])['questions_id'].transform('count')
df_merge['num_ans_per_ques'] = df_merge.groupby(['questions_id'])['answers_id'].transform('count')
df_merge['num_tags_professional'] = df_merge['professionals_tag_name'].str.split(",").str.len()
df_merge['num_tags_question'] = df_merge['questions_tag_name'].str.split(",").str.len()

**Additional line**: Convert dtype

In [10]:
df_merge['answers_date_added'] = pd.to_datetime(df_merge['answers_date_added']).dt.strftime('%Y-%m-%d %H:%M:%S')

In [11]:
df_merge.head()

,answers_id,answers_author_id,answers_question_id,answers_date_added,answers_body,answers_id_num,questions_id,questions_author_id,questions_date_added,questions_title,...,professionals_date_joined,professionals_id_num,tag_users_user_id,professionals_tag_name,id_y,score_y,num_of_ans_by_professional,num_ans_per_ques,num_tags_professional,num_tags_question
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14,<p>Hi!</p>\n<p>You are asking a very interesti...,0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26+00:00,Teacher career question,...,2015-10-19 20:56:49+00:00,2410,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",332a511f1569444485cf7a7a556a5e54,1,1710,1,12.0,3.0
1,f3519ab99a1a4a13a8a9ecb814287d2a,36ff3b3666df400f956f8335cf53e09e,0f1d6a4f276c4a05878dd48e03e52289,2016-07-31 15:35:54,<p>Hi Rodrigo!</p>\n<p>The important thing to ...,11,0f1d6a4f276c4a05878dd48e03e52289,585ac233015447cc9e9a217044e515e1,2016-05-19 22:16:25+00:00,what kind of college could i go to for a soc...,...,2015-10-19 20:56:49+00:00,2410,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",0f1d6a4f276c4a05878dd48e03e52289,1,1710,1,12.0,3.0
2,825f6e316a5f48328d6f8af831df9940,36ff3b3666df400f956f8335cf53e09e,0149c6d63e214040b44d4a3789bb00ba,2018-04-15 23:08:46,<p>Congratulations on being interested in find...,71,0149c6d63e214040b44d4a3789bb00ba,34217a1861d640a58c85e033414cf9cb,2018-04-12 17:13:45+00:00,What is the best way to prepare for studying e...,...,2015-10-19 20:56:49+00:00,2410,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",0149c6d63e214040b44d4a3789bb00ba,2,1710,2,12.0,3.0
3,fb2c794175304c4caeb55e654270421f,a32736b04c27437da3078374d47af1b1,0149c6d63e214040b44d4a3789bb00ba,2018-04-13 18:18:05,<p>Hi Elisabeth! </p><p><br></p><p>If you are ...,72,0149c6d63e214040b44d4a3789bb00ba,34217a1861d640a58c85e033414cf9cb,2018-04-12 17:13:45+00:00,What is the best way to prepare for studying e...,...,2018-04-13 17:48:09+00:00,18373,a32736b04c27437da3078374d47af1b1,computer-software,0149c6d63e214040b44d4a3789bb00ba,2,1,2,1.0,3.0
4,f3fc23809cda472780fc565334f35000,36ff3b3666df400f956f8335cf53e09e,acc611cfb5c44daa8a3d7d65dfffa5ff,2018-08-14 10:37:01,<p>The most important thing that you can do is...,102,acc611cfb5c44daa8a3d7d65dfffa5ff,5b751a8ee4a047f7a08ce9eb5e43e5a2,2018-08-14 04:49:33+00:00,How should I prepare myself for my job search ...,...,2015-10-19 20:56:49+00:00,2410,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",acc611cfb5c44daa8a3d7d65dfffa5ff,1,1710,1,12.0,4.0


In [12]:
print("Maximum number of answer per question : " + str(df_merge['num_ans_per_ques'].max()))
print("Maximum number of tags per professional : " + str(df_merge['num_tags_professional'].max()))
print("Maximum number of tags per question : " + str(df_merge['num_tags_question'].max()))

Maximum number of answer per question : 58
Maximum number of tags per professional : 82.0
Maximum number of tags per question : 54.0


**Merge answered questions tags with professional's tag**

In [13]:
# Merge professionals previous answered
# questions tags into professionals tags

# select professionals answered questions tags
# and stored as a dataframe
professionals_prev_ans_tags = df_merge[['professionals_id', 'questions_tag_name']]

# drop nulss values from that
professionals_prev_ans_tags = professionals_prev_ans_tags.dropna()

# because professionals answers multiple questions,
# we group all of tags of each user into single row
professionals_prev_ans_tags = professionals_prev_ans_tags.groupby(['professionals_id'])['questions_tag_name'].apply(','.join).reset_index()

# drop duplicates tags from each professionals rows
professionals_prev_ans_tags['questions_tag_name'] = (
    professionals_prev_ans_tags['questions_tag_name'].str.split(',').apply(set).str.join(',')
)

# finally merge the dataframe with professionals dataframe
df_professionals = df_professionals.merge(professionals_prev_ans_tags, how='left', on='professionals_id')

# join professionals tags and their answered tags
# we replace nan values with ""
df_professionals['professional_all_tags'] = (
    df_professionals[['professionals_tag_name', 'questions_tag_name']].apply(
        lambda x: ','.join(x.dropna()),
        axis=1
    )
)

**Handling null and duplicates values**

In [14]:
# handling null values 
df_questions['score'] = df_questions['score'].fillna(0)
df_questions['score'] = df_questions['score'].astype(int)
df_questions['questions_tag_name'] = df_questions['questions_tag_name'].fillna('No Tag')

# remove duplicates tags from each questions 
df_questions['questions_tag_name'] = df_questions['questions_tag_name'].str.split(',').apply(set).str.join(',')

# fill nan with 'No Tag' if any
df_professionals['professional_all_tags'] = df_professionals['professional_all_tags'].fillna('No Tag')

# replace "" with "No Tag", because previously we replace nan with ""
df_professionals['professional_all_tags'] = df_professionals['professional_all_tags'].replace('', 'No Tag')
df_professionals['professionals_location'] = df_professionals['professionals_location'].fillna('No Location')
df_professionals['professionals_industry'] = df_professionals['professionals_industry'].fillna('No Industry')

# remove duplicates tags from each professionals 
df_professionals['professional_all_tags'] = df_professionals['professional_all_tags'].str.split(',').apply(set).str.join(',')

# remove some null values from df_merge
df_merge['num_ans_per_ques']  = df_merge['num_ans_per_ques'].fillna(0)
df_merge['num_tags_professional'] = df_merge['num_tags_professional'].fillna(0)
df_merge['num_tags_question'] = df_merge['num_tags_question'].fillna(0)

In [15]:
df_merge.head()

,answers_id,answers_author_id,answers_question_id,answers_date_added,answers_body,answers_id_num,questions_id,questions_author_id,questions_date_added,questions_title,...,professionals_date_joined,professionals_id_num,tag_users_user_id,professionals_tag_name,id_y,score_y,num_of_ans_by_professional,num_ans_per_ques,num_tags_professional,num_tags_question
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14,<p>Hi!</p>\n<p>You are asking a very interesti...,0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26+00:00,Teacher career question,...,2015-10-19 20:56:49+00:00,2410,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",332a511f1569444485cf7a7a556a5e54,1,1710,1,12.0,3.0
1,f3519ab99a1a4a13a8a9ecb814287d2a,36ff3b3666df400f956f8335cf53e09e,0f1d6a4f276c4a05878dd48e03e52289,2016-07-31 15:35:54,<p>Hi Rodrigo!</p>\n<p>The important thing to ...,11,0f1d6a4f276c4a05878dd48e03e52289,585ac233015447cc9e9a217044e515e1,2016-05-19 22:16:25+00:00,what kind of college could i go to for a soc...,...,2015-10-19 20:56:49+00:00,2410,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",0f1d6a4f276c4a05878dd48e03e52289,1,1710,1,12.0,3.0
2,825f6e316a5f48328d6f8af831df9940,36ff3b3666df400f956f8335cf53e09e,0149c6d63e214040b44d4a3789bb00ba,2018-04-15 23:08:46,<p>Congratulations on being interested in find...,71,0149c6d63e214040b44d4a3789bb00ba,34217a1861d640a58c85e033414cf9cb,2018-04-12 17:13:45+00:00,What is the best way to prepare for studying e...,...,2015-10-19 20:56:49+00:00,2410,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",0149c6d63e214040b44d4a3789bb00ba,2,1710,2,12.0,3.0
3,fb2c794175304c4caeb55e654270421f,a32736b04c27437da3078374d47af1b1,0149c6d63e214040b44d4a3789bb00ba,2018-04-13 18:18:05,<p>Hi Elisabeth! </p><p><br></p><p>If you are ...,72,0149c6d63e214040b44d4a3789bb00ba,34217a1861d640a58c85e033414cf9cb,2018-04-12 17:13:45+00:00,What is the best way to prepare for studying e...,...,2018-04-13 17:48:09+00:00,18373,a32736b04c27437da3078374d47af1b1,computer-software,0149c6d63e214040b44d4a3789bb00ba,2,1,2,1.0,3.0
4,f3fc23809cda472780fc565334f35000,36ff3b3666df400f956f8335cf53e09e,acc611cfb5c44daa8a3d7d65dfffa5ff,2018-08-14 10:37:01,<p>The most important thing that you can do is...,102,acc611cfb5c44daa8a3d7d65dfffa5ff,5b751a8ee4a047f7a08ce9eb5e43e5a2,2018-08-14 04:49:33+00:00,How should I prepare myself for my job search ...,...,2015-10-19 20:56:49+00:00,2410,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",acc611cfb5c44daa8a3d7d65dfffa5ff,1,1710,1,12.0,4.0


# Building model in LightFM
In this steps, we are going to build our lightFM model using lightFM python library. Firstly, we have to create lightFM `Dataset` for our model. LightFM Dataset class makes it really easy for us for creating `interaction matrix`, `weights` and `user/item features`.

* `interaction matrix`: it is a matrix that contains user/item interactions or professional/question interactions.
* `weights`: weight of interaction matrix. Less weight means less importance to that interaction matrix
* `user/item features`: user/item features supplied as like this `(user_id, ['feature_1', 'feature_2', 'feature_3'])

If you want to how lightFM python library's dataset class works and how to use it, please go to this link [Building LightFM Datasets](http://https://lyst.github.io/lightfm/docs/examples/dataset.html).

Then, after that we will be start building our lightFM model using LightFM class. LightFM class makes it really easy for making lightFM model. After that we will train our model by our data.

**Creating features list for Dataset class**

In [16]:
# generating features list for mapping
question_feature_list = generate_feature_list(
    df_questions,
    ['questions_tag_name'])

professional_feature_list = generate_feature_list(
    df_professionals,
    ['professional_all_tags'])

In [17]:
question_feature_list

0                   professor
1                     lecture
2                     college
3                        army
4                    military
                 ...         
77191         law-enforcement
77192                    java
77193        computer-science
77194    computer-engineering
77195             programming
Length: 77196, dtype: object

In [18]:
professional_feature_list

0                                        consulting
1                                            resume
2                                            No Tag
3                                              bain
4                                         emotional
                            ...                    
228831                                    mentoring
228832    indirect-sales-(channel)-and-direct-sales
228833                          women-in-leadership
228834                              contact-centers
228835                               communications
Length: 228836, dtype: object

In [19]:
# calculate our weight value
df_merge['total_weights'] = 1 / (
    df_merge['num_ans_per_ques']
)

# creating features for feeding into lightfm
df_questions['question_features'] = create_features(
    df_questions, ['questions_tag_name'],
    'questions_id_num'
)

df_professionals['professional_features'] = create_features(
    df_professionals,
    ['professional_all_tags'],
    'professionals_id_num'
)

In [20]:
df_questions.head()

,questions_id,questions_author_id,questions_date_added,questions_title,questions_body,questions_id_num,tag_questions_question_id,questions_tag_name,id,score,students_id,students_location,students_date_joined,students_id_num,question_features
0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26+00:00,Teacher career question,What is a maths teacher? what is a ma...,0,332a511f1569444485cf7a7a556a5e54,"professor,lecture,college",332a511f1569444485cf7a7a556a5e54,1,8f6f374ffd834d258ab69d376dd998f5,"Coimbatore, Tamil Nadu, India",2016-04-22 10:07:32+00:00,6890.0,"(0, [professor, lecture, college])"
1,eb80205482e4424cad8f16bc25aa2d9c,acccbda28edd4362ab03fb8b6fd2d67b,2016-05-20 16:48:25+00:00,I want to become an army officer. What can I d...,I am Priyanka from Bangalore . Now am in 10th ...,1,eb80205482e4424cad8f16bc25aa2d9c,"army,military",eb80205482e4424cad8f16bc25aa2d9c,5,acccbda28edd4362ab03fb8b6fd2d67b,"Providence, Rhode Island",2016-05-20 16:29:08+00:00,10189.0,"(1, [army, military])"
2,4ec31632938a40b98909416bdd0decff,f2c179a563024ccc927399ce529094b5,2017-02-08 19:13:38+00:00,Will going abroad for your first job increase ...,I'm planning on going abroad for my first job....,2,4ec31632938a40b98909416bdd0decff,"working-abroad,overseas",4ec31632938a40b98909416bdd0decff,2,f2c179a563024ccc927399ce529094b5,NaN,2017-02-07 15:51:57+00:00,18023.0,"(2, [working-abroad, overseas])"
3,2f6a9a99d9b24e5baa50d40d0ba50a75,2c30ffba444e40eabb4583b55233a5a4,2017-09-01 14:05:32+00:00,To become a specialist in business management...,i hear business management is a hard way to ge...,3,2f6a9a99d9b24e5baa50d40d0ba50a75,"networking,business",2f6a9a99d9b24e5baa50d40d0ba50a75,2,2c30ffba444e40eabb4583b55233a5a4,"North Lauderdale, Florida",2017-09-01 14:02:02+00:00,20803.0,"(3, [networking, business])"
4,5af8880460c141dbb02971a1a8369529,aa9eb1a2ab184ebbb00dc01ab663428a,2017-09-01 02:36:54+00:00,Are there any scholarships out there for stude...,I'm trying to find scholarships for first year...,4,5af8880460c141dbb02971a1a8369529,"firstgeneration,scholarships,highschoolsenior,...",5af8880460c141dbb02971a1a8369529,2,aa9eb1a2ab184ebbb00dc01ab663428a,"Tunnel Hill, Georgia",2017-09-01 02:29:06+00:00,20505.0,"(4, [firstgeneration, scholarships, highschool..."


In [21]:
df_professionals.head()

,professionals_id,professionals_location,professionals_industry,professionals_headline,professionals_date_joined,professionals_id_num,tag_users_user_id,professionals_tag_name,questions_tag_name,professional_all_tags,professional_features
0,9ced4ce7519049c0944147afb75a8ce3,No Location,No Industry,NaN,2011-10-05 20:35:19+00:00,0,NaN,NaN,"consulting,resume","consulting,resume","(0, [consulting, resume])"
1,f718dcf6d2ec4cb0a52a9db59d7f9e67,No Location,No Industry,NaN,2011-10-05 20:49:21+00:00,1,NaN,NaN,NaN,No Tag,"(1, [No Tag])"
2,0c673e046d824ec0ad0ebe012a0673e4,"New York, New York",No Industry,NaN,2011-10-18 17:31:26+00:00,2,0c673e046d824ec0ad0ebe012a0673e4,"consulting,consulting,consulting,consulting,co...","bain,emotional,residency,entrepreneurship,scho...","bain,emotional,residency,entrepreneurship,air-...","(2, [bain, emotional, residency, entrepreneurs..."
3,977428d851b24183b223be0eb8619a8c,"Boston, Massachusetts",No Industry,NaN,2011-11-09 20:39:29+00:00,3,NaN,NaN,"police,law,fashion-design,law-enforcement,crim...","fashion-design,police,law,law-enforcement,crim...","(3, [fashion-design, police, law, law-enforcem..."
4,e2d57e5041a44f489288397c9904c2b2,No Location,No Industry,NaN,2011-12-10 22:14:44+00:00,4,NaN,NaN,NaN,No Tag,"(4, [No Tag])"


**LightFM Dataset**: In this steps we are going to build lightfm datasets. And then we will be building our ineractions matrix, weights and professional/question features.

In [22]:
question_feature_list

0                   professor
1                     lecture
2                     college
3                        army
4                    military
                 ...         
77191         law-enforcement
77192                    java
77193        computer-science
77194    computer-engineering
77195             programming
Length: 77196, dtype: object

In [23]:
professional_feature_list

0                                        consulting
1                                            resume
2                                            No Tag
3                                              bain
4                                         emotional
                            ...                    
228831                                    mentoring
228832    indirect-sales-(channel)-and-direct-sales
228833                          women-in-leadership
228834                              contact-centers
228835                               communications
Length: 228836, dtype: object

In [24]:
len(set(df_professionals['professionals_id_num']))

28152

In [25]:
len(set(df_questions['questions_id_num']))

23931

In [26]:
df_merge.head()

,answers_id,answers_author_id,answers_question_id,answers_date_added,answers_body,answers_id_num,questions_id,questions_author_id,questions_date_added,questions_title,...,professionals_id_num,tag_users_user_id,professionals_tag_name,id_y,score_y,num_of_ans_by_professional,num_ans_per_ques,num_tags_professional,num_tags_question,total_weights
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14,<p>Hi!</p>\n<p>You are asking a very interesti...,0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26+00:00,Teacher career question,...,2410,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",332a511f1569444485cf7a7a556a5e54,1,1710,1,12.0,3.0,1.0
1,f3519ab99a1a4a13a8a9ecb814287d2a,36ff3b3666df400f956f8335cf53e09e,0f1d6a4f276c4a05878dd48e03e52289,2016-07-31 15:35:54,<p>Hi Rodrigo!</p>\n<p>The important thing to ...,11,0f1d6a4f276c4a05878dd48e03e52289,585ac233015447cc9e9a217044e515e1,2016-05-19 22:16:25+00:00,what kind of college could i go to for a soc...,...,2410,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",0f1d6a4f276c4a05878dd48e03e52289,1,1710,1,12.0,3.0,1.0
2,825f6e316a5f48328d6f8af831df9940,36ff3b3666df400f956f8335cf53e09e,0149c6d63e214040b44d4a3789bb00ba,2018-04-15 23:08:46,<p>Congratulations on being interested in find...,71,0149c6d63e214040b44d4a3789bb00ba,34217a1861d640a58c85e033414cf9cb,2018-04-12 17:13:45+00:00,What is the best way to prepare for studying e...,...,2410,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",0149c6d63e214040b44d4a3789bb00ba,2,1710,2,12.0,3.0,0.5
3,fb2c794175304c4caeb55e654270421f,a32736b04c27437da3078374d47af1b1,0149c6d63e214040b44d4a3789bb00ba,2018-04-13 18:18:05,<p>Hi Elisabeth! </p><p><br></p><p>If you are ...,72,0149c6d63e214040b44d4a3789bb00ba,34217a1861d640a58c85e033414cf9cb,2018-04-12 17:13:45+00:00,What is the best way to prepare for studying e...,...,18373,a32736b04c27437da3078374d47af1b1,computer-software,0149c6d63e214040b44d4a3789bb00ba,2,1,2,1.0,3.0,0.5
4,f3fc23809cda472780fc565334f35000,36ff3b3666df400f956f8335cf53e09e,acc611cfb5c44daa8a3d7d65dfffa5ff,2018-08-14 10:37:01,<p>The most important thing that you can do is...,102,acc611cfb5c44daa8a3d7d65dfffa5ff,5b751a8ee4a047f7a08ce9eb5e43e5a2,2018-08-14 04:49:33+00:00,How should I prepare myself for my job search ...,...,2410,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",acc611cfb5c44daa8a3d7d65dfffa5ff,1,1710,1,12.0,4.0,1.0


In [27]:
len(
    list(
        zip(
            df_merge['professionals_id_num'],
            df_merge['questions_id_num'],
            df_merge['total_weights']
        )
    )
)

50098

In [28]:
# Dataset building for lightFM

# define our dataset variable
# then we feed unique professionals and questions ids
# and item and professional feature list
# this will create lightfm internel mapping
dataset = Dataset()
dataset.fit(
    set(df_professionals['professionals_id_num']),
    set(df_questions['questions_id_num']),
    item_features=question_feature_list,
    user_features=professional_feature_list
)

# now we are building ineractions matrix between professionals and questions.
# we are passing professional and question id as a tuple
# e.g -> pd.Series ((pro_id, question_id), (pro_id, question_id))
# then we use lightFM build in method for building interactions matrix
df_merge['author_question_id_tuple'] = list(
    zip(
        df_merge['professionals_id_num'],
        df_merge['questions_id_num'],
        df_merge['total_weights']
    )
)

interactions, weights = dataset.build_interactions(
    df_merge['author_question_id_tuple']
)

# now we are building our questions and professionals features
# in a way that lightFM understand.
# we are using lightFM build in method for building
# questions and professionals features
questions_features = dataset.build_item_features(
    df_questions['question_features']
)

professional_features = dataset.build_user_features(
    df_professionals['professional_features']
)

**Model building and training**

In [29]:
# Model building part

# define lightFm model by specifying hyper-parameter
# then fit the model with interactions matrix, item and user features
model = LightFM(
    no_components=150,
    learning_rate=0.05,
    loss='warp',
    random_state=2019
)

model.fit(
    interactions,
    item_features=questions_features,
    user_features=professional_features,
    sample_weight=weights,
    epochs=5,
    num_threads=4,
    verbose=True
)

Epoch: 100%|██████████| 5/5 [00:16<00:00,  3.33s/it]


# Evaluating the performance of the model 
Now we have to evaluate our model to see it's performance. No matter how good your model is, if you can't evaluate your model correctly you can't imporove and trust your model. For recommendation problem, there is not very good matrics for evaluating. But luckily lightfm provides us a very rich set of evaluating matrics. In this steps, we will be calculating AUC scores for our model.

**What is AUC score in lightfm library?**: It measure the ROC AUC metric for a model: the probability that a randomly chosen positive example has a higher score than a randomly chosen negative example. A perfect score is 1.0. 

Let's see what is our model score. 

In [30]:
calculate_auc_score(model, interactions, questions_features, professional_features)

0.9149836

Wow! That is really impresive. Over AUC is over 90 percent. That is really excellent. This tells us that the quality of our overall model is very good.

**Make real recommendations**: Now we already see how our model is by looking at AUC score. But now let's see some real example of recommendation.

In [31]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

def recommend_questions(professional_ids):
    for professional in professional_ids:
        # print their previous answered question title
        previous_q_id_num = df_merge.loc[df_merge['professionals_id_num'] == professional][:3]['questions_id_num']
        df_previous_questions = df_questions.loc[df_questions['questions_id_num'].isin(previous_q_id_num)]
        print('Professional Id (' + str(professional) + "): Previous Answered Questions")
        display_side_by_side(
            df_previous_questions[['questions_title', 'question_features']],
            df_professionals.loc[df_professionals.professionals_id_num == professional][['professionals_id_num','professionals_tag_name']])
        
        # predict
        discard_qu_id = df_previous_questions['questions_id_num'].values.tolist()
        df_use_for_prediction = df_questions.loc[~df_questions['questions_id_num'].isin(discard_qu_id)]
        questions_id_for_predict = df_use_for_prediction['questions_id_num'].values.tolist()
        
        scores = model.predict(
            professional,
            questions_id_for_predict,
            item_features=questions_features,
            user_features=professional_features)
        
        df_use_for_prediction['scores'] = scores
        df_use_for_prediction = df_use_for_prediction.sort_values(by='scores', ascending=False)[:8]
        print('Professional Id (' + str(professional) + "): Recommended Questions: ")
        display(df_use_for_prediction[['questions_title', 'question_features']])

In [32]:
recommend_questions([1200, 19897, 3])

Professional Id (1200): Previous Answered Questions


,questions_title,question_features
,professionals_id_num,professionals_tag_name
1200,1200,"marketing,strategy,entrepreneurship,management,java,advertising,python,data-analysis,online-advertising,real-estate,team-leadership,dj,analytics,display-advertising,football,blackjack,hip-hop,billiards,break"


Professional Id (1200): Recommended Questions: 


,questions_title,question_features
20217,What are the most common and uncommon battles ...,"(20217, [entrepreneurship, business])"
17876,Is it worth it to pursue an advanced degree in...,"(17876, [marketing, business])"
19011,How do you get started in starting your own bu...,"(19011, [marketing, business, management])"
10191,How do you become the youngest female CEO of a...,"(10191, [entrepreneurship, business])"
15671,How beneficial will a business major be ?,"(15671, [finance, marketing, accountant, busin..."
10073,How important is a business degree when trying...,"(10073, [entrepreneurship, business])"
20431,What are the average income for someone that w...,"(20431, [entrepreneurship, business])"
13378,As someone who one day wants to open up his ow...,"(13378, [marketing, finance, sales, entreprene..."


Professional Id (19897): Previous Answered Questions


...

,questions_title,question_features
2310,what is one of best things about being an anim...,"(2310, [design, artist, animation, art])"
9682,How to get started in animation?,"(9682, [artist, animation, art])"
19407,How can you be a successful photographer? What...,"(19407, [art, graphic-design, photography])"
6058,How should you start in the Graphic Design ind...,"(6058, [design, graphic-design, art])"
1416,Is Competition In The Animation Field Low or H...,"(1416, [art, animation])"
1203,What is a good app to use for animating?,"(1203, [art, animation])"
17325,what are the required fields forgraphic design?,"(17325, [art, graphic-design])"
13484,Would a Graphic Design degree be a feesible op...,"(13484, [art, graphic-design])"


Professional Id (3): Previous Answered Questions


,questions_title,question_features
11339,What are the different jobs a person can do in Forensic Science?,"(11339, [criminal, science, forensic, justice])"
14818,What does a typical work day for a forensic scientist look like?,"(14818, [No Tag])"
19077,Is most of your day spent working when being a detective?,"(19077, [detective])"
,professionals_id_num,professionals_tag_name
3,3,NaN


Professional Id (3): Recommended Questions: 


,questions_title,question_features
2423,How long does it take to become a Detective?,"(2423, [police, law, law-enforcement, criminal..."
17184,What types of Detectives are there?,"(17184, [police, law, law-enforcement, crimina..."
1941,What are important characteristics of a lawyer?,"(1941, [law-enforcement, law-school, law, lawy..."
18124,what is the starting salary for a police offic...,"(18124, [law, law-enforcement])"
16214,"Do you go to college, then B.L.E.T( Basic Law ...","(16214, [police, law, law-enforcement])"
9778,I want to be a police officer or a police disp...,"(9778, [police, police-officer, law, law-enfor..."
18947,"Could I go straight into Law Enforcememt, when...","(18947, [police, law, law-enforcement])"
20902,How would i get into a law school?,"(20902, [law, law-enforcement])"


**Analysis**: Awesome! Finally we can see our recommendation by our model. Let's take some time to ponder over the recommendations.

* For the first professional(1200) has not answer any questions yet. But he/she follows some tags. Our model take those tags as features and predict questions that has similar tags.
* For the second professionals(19897) answered one questions that has tag major. But in his profile he follows tags like creative works like arts, illurstrator etc. So our model recommend questions that has creative tags like arts, illustrator because he follows more tags one creative works.
* For the third professionals(3): answered questions that has tag forensic, criminal, science, justice, detective. From the tags we can get an idea of professionals interests. Our model also learn that. That's why it recommend items that has tags like law, criminal, detective.

This is just a simple exploration. Hope you get idea of the model recommendations. The model can survive cold-start, high-popularity problem. It also recommend those questions that has less answer because of its weights that I provied during traning. Now we build our model and tested it. In the next section, we will look how we can put this model in production.

# Model in Production
We previously saw how lightfm model works and build for this project. Well, now we are going build a pipeline that will help us for putting this model into production. We are going to build class for each steps discuss in step 2. Also, we are going to build some additional functions and methods that will add additional functionality to the model. 

Here is the picture of our pipeline: 
![](https://i.imgur.com/Kh4rVcL.png)


We will now build class for each of these steps. Without further do let's begin. 

In [33]:
# Read all our datasets again
# and store them in pandas dataframe objects.
base_path = './input/'
df_answer_scores = pd.read_csv(
    base_path + 'answer_scores.csv')

df_answers = pd.read_csv(
    base_path + 'answers.csv',
    parse_dates=['answers_date_added'])

df_comments = pd.read_csv(
    base_path + 'comments.csv')

df_emails = pd.read_csv(
    base_path + 'emails.csv')

df_group_memberships = pd.read_csv(
    base_path + 'group_memberships.csv')

df_groups = pd.read_csv(
    base_path + 'groups.csv')

df_matches = pd.read_csv(
    base_path + 'matches.csv')

df_professionals = pd.read_csv(
    base_path + 'professionals.csv',
    parse_dates=['professionals_date_joined'])

df_question_scores = pd.read_csv(
    base_path + 'question_scores.csv')

df_questions = pd.read_csv(
    base_path + 'questions.csv',
    parse_dates=['questions_date_added'])

df_school_memberships = pd.read_csv(
    base_path + 'school_memberships.csv')

df_students = pd.read_csv(
    base_path + 'students.csv',
    parse_dates=['students_date_joined'])

df_tag_questions = pd.read_csv(
    base_path + 'tag_questions.csv')

df_tag_users = pd.read_csv(
    base_path + 'tag_users.csv')

df_tags = pd.read_csv(
    base_path + 'tags.csv')

**Data Processing Class**: Now we are going to build a class that will be used for data cleaning and processing specificly designed for CareerVillage Datasetes. I have provided details document and comment with each part of the code. This will help understanding the code and intention very well.

In [34]:
class CareerVillageDataPreparation:
    """
    Clean and process data CareerVillage Data. 
    
    This class process data in a way that will be useful
    for building lightFM dataset. 
    """
    
    def __init__(self):
        pass

    def _assign_unique_id(self, data, id_col_name):
        """
        Generate unique integer id for users, questions and answers

        Parameters
        ----------
        data: Dataframe
            Pandas Dataframe for Users or Q&A. 
        id_col_name : String 
            New integer id's column name.

        Returns
        -------
        Dataframe
            Updated dataframe containing new id column
        """
        new_dataframe=data.assign(
            int_id_col_name=np.arange(len(data))
            ).reset_index(drop=True)
        return new_dataframe.rename(columns={'int_id_col_name': id_col_name})

    def _dropna(self, data, column, axis):
        """Drop null values from specific column"""
        return data.dropna(column, axis=axis)

    def _merge_data(self, left_data, left_key, right_data, right_key, how):
        """
        This function is used for merging two dataframe.
        
        Parameters
        -----------
        left_data: Dataframe
            Left side dataframe for merge
        left_key: String
            Left Dataframe merge key
        right_data: Dataframe
            Right side dataframe for merge
        right_key: String
            Right Dataframe merge key
        how: String
            Method of merge (inner, left, right, outer)
            
        
        Returns
        --------
        Dataframe
            A new dataframe merging left and right dataframe
        """
        return left_data.merge(
            right_data,
            how=how,
            left_on=left_key,
            right_on=right_key)

    def _group_tags(self, data, group_by, tag_column):
        """Grouop multiple tags into single rows sepearated by comma"""
        return data.groupby(
            [group_by])[tag_column].apply(
            ','.join).reset_index()

    def _merge_cv_datasets(
        self,
        professionals,students,
        questions,answers,
        tags,tag_questions,tag_users, questions_score):
        """
        This function merges all the necessary 
        CareerVillage dataset in defined way. 
        
        Parameters
        ------------
        professionals,students,
        questions,answers,
        tags,tag_questions,
        tag_users,
        questions_score: Dataframe
            Pandas dataframe defined by it's name
        
        
        Returns
        ---------
        questions, professionals: Dataframe
            Updated dataframe after merge
        merge: Dataframe
            A new datframe after merging answers with questions
        """
        
        
        # merge tag_questions with tags name
        # then group all tags for each question into single rows
        tag_question = self._merge_data(
            left_data=tag_questions,
            left_key='tag_questions_tag_id',
            right_data=tags,
            right_key='tags_tag_id',
            how='inner')
        tag_question = self._group_tags(
            data=tag_question,
            group_by='tag_questions_question_id',
            tag_column='tags_tag_name')
        
        tag_question = tag_question.rename(
            columns={'tags_tag_name': 'questions_tag_name'})
        
        # merge tag_users with tags name
        # then group all tags for each user into single rows 
        # after that rename the tag column name
        tags_pro = self._merge_data(
            left_data=tag_users,
            left_key='tag_users_tag_id',
            right_data=tags,
            right_key='tags_tag_id',
            how='inner')
        tags_pro = self._group_tags(
            data=tags_pro,
            group_by='tag_users_user_id',
            tag_column='tags_tag_name')
        tags_pro = tags_pro.rename(
            columns={'tags_tag_name': 'professionals_tag_name'})
        
        # merge professionals and questions tags with main merge_dataset 
        questions = self._merge_data(
            left_data=questions,
            left_key='questions_id',
            right_data=tag_question,
            right_key='tag_questions_question_id',
            how='left')
        professionals = self._merge_data(
            left_data=professionals,
            left_key='professionals_id',
            right_data=tags_pro,
            right_key='tag_users_user_id',
            how='left')
        
        # merge questions with scores 
        questions = self._merge_data(
            left_data=questions,
            left_key='questions_id',
            right_data=questions_score,
            right_key='id',
            how='left')
        
        # merge questions with students
        questions = self._merge_data(
            left_data=questions,
            left_key='questions_author_id',
            right_data=students,
            right_key='students_id',
            how='left')
        
        # merge answers with questions
        # then merge professionals and questions score with that
        merge = self._merge_data(
            left_data=answers,
            left_key='answers_question_id',
            right_data=questions,
            right_key='questions_id',
            how='inner')
        
        merge = self._merge_data(
            left_data=merge,
            left_key='answers_author_id',
            right_data=professionals,
            right_key='professionals_id',
            how='inner')
        return questions, professionals, merge

    def _drop_duplicates_tags(self, data, col_name):
        # drop duplicates tags from each row
        return (
            data[col_name].str.split(
                ',').apply(set).str.join(','))

    def _merge_pro_pre_ans_tags(self, professionals, merge):
        ########################
        # Merge professionals previous answered
        # questions tags into professionals tags
        ########################
        
        # select professionals answered questions tags
        # and stored as a dataframe
        professionals_prev_ans_tags = (
            merge[['professionals_id', 'questions_tag_name']])
        # drop null values from that
        professionals_prev_ans_tags = professionals_prev_ans_tags.dropna()
        
        # because professsionals answers multiple questions,
        # we group all of tags of each user into single row
        professionals_prev_ans_tags = self._group_tags(
            data=professionals_prev_ans_tags,
            group_by='professionals_id',
            tag_column='questions_tag_name')
        
        # drop duplicates tags from each professionals rows
        professionals_prev_ans_tags['questions_tag_name'] = \
        self._drop_duplicates_tags(
            professionals_prev_ans_tags, 'questions_tag_name')
        
        # finally merge the dataframe with professionals dataframe
        professionals = self._merge_data(
            left_data=professionals,
            left_key='professionals_id',
            right_data=professionals_prev_ans_tags,
            right_key='professionals_id',
            how='left')
        
        # join professionals tags and their answered tags 
        # we replace nan values with ""
        professionals['professional_all_tags'] = (
            professionals[['professionals_tag_name',
                           'questions_tag_name']].apply(
                lambda x: ','.join(x.dropna()),
                axis=1))
        return professionals

    def prepare(
        self,
        professionals,students,
        questions,answers,
        tags,tag_questions,tag_users, questions_score):
        
        """
        This function clean and process 
        CareerVillage Data sets. 
        """
        
        # assign unique integer id
        professionals = self._assign_unique_id(
            professionals, 'professionals_id_num')
        students = self._assign_unique_id(
            students, 'students_id_num')
        questions = self._assign_unique_id(
            questions, 'questions_id_num')
        answers = self._assign_unique_id(
            answers, 'answers_id_num')
        
        # just dropna from tags 
        tags = tags.dropna()
        tags['tags_tag_name'] = tags['tags_tag_name'].str.replace(
            '#', '')
        
        # merge necessary datasets
        df_questions, df_professionals, df_merge = self._merge_cv_datasets(
            professionals,students,
            questions,answers,
            tags,tag_questions,tag_users,
            questions_score)
        
        # Generate some features for calculates weights
        # that will use with interaction matrix
        df_merge['num_ans_per_ques'] = df_merge.groupby(
            ['questions_id'])['answers_id'].transform('count')
        
        # merge pro previoius answered question tags with pro tags 
        df_professionals = self._merge_pro_pre_ans_tags(
            df_professionals, df_merge)
        
        # some more pre-processing 
        # handling null values 
        df_questions['score'] = df_questions['score'].fillna(0)
        df_questions['score'] = df_questions['score'].astype(int)
        df_questions['questions_tag_name'] = \
        df_questions['questions_tag_name'].fillna('No Tag')
        
        # remove duplicates tags from each questions 
        df_questions['questions_tag_name'] = \
        df_questions['questions_tag_name'].str.split(
            ',').apply(set).str.join(',')

        # fill nan with 'No Tag' if any 
        df_professionals['professional_all_tags'] = \
        df_professionals['professional_all_tags'].fillna(
            'No Tag')
        # replace "" with "No Tag", because previously we replace nan with ""
        df_professionals['professional_all_tags'] = \
        df_professionals['professional_all_tags'].replace(
            '', 'No Tag')
        
        df_professionals['professionals_location'] = \
        df_professionals['professionals_location'].fillna(
            'No Location')
        
        df_professionals['professionals_industry'] = \
        df_professionals['professionals_industry'].fillna(
            'No Industry')

        # remove duplicates tags from each professionals
        df_professionals['professional_all_tags'] = \
        df_professionals['professional_all_tags'].str.split(
            ',').apply(set).str.join(',')

        # remove some null values from df_merge
        df_merge['num_ans_per_ques']  = \
        df_merge['num_ans_per_ques'].fillna(0)
        
        return df_questions, df_professionals, df_merge

In [35]:
df_merge.head()

,answers_id,answers_author_id,answers_question_id,answers_date_added,answers_body,answers_id_num,questions_id,questions_author_id,questions_date_added,questions_title,...,tag_users_user_id,professionals_tag_name,id_y,score_y,num_of_ans_by_professional,num_ans_per_ques,num_tags_professional,num_tags_question,total_weights,author_question_id_tuple
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14,<p>Hi!</p>\n<p>You are asking a very interesti...,0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26+00:00,Teacher career question,...,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",332a511f1569444485cf7a7a556a5e54,1,1710,1,12.0,3.0,1.0,"(2410, 0, 1.0)"
1,f3519ab99a1a4a13a8a9ecb814287d2a,36ff3b3666df400f956f8335cf53e09e,0f1d6a4f276c4a05878dd48e03e52289,2016-07-31 15:35:54,<p>Hi Rodrigo!</p>\n<p>The important thing to ...,11,0f1d6a4f276c4a05878dd48e03e52289,585ac233015447cc9e9a217044e515e1,2016-05-19 22:16:25+00:00,what kind of college could i go to for a soc...,...,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",0f1d6a4f276c4a05878dd48e03e52289,1,1710,1,12.0,3.0,1.0,"(2410, 7, 1.0)"
2,825f6e316a5f48328d6f8af831df9940,36ff3b3666df400f956f8335cf53e09e,0149c6d63e214040b44d4a3789bb00ba,2018-04-15 23:08:46,<p>Congratulations on being interested in find...,71,0149c6d63e214040b44d4a3789bb00ba,34217a1861d640a58c85e033414cf9cb,2018-04-12 17:13:45+00:00,What is the best way to prepare for studying e...,...,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",0149c6d63e214040b44d4a3789bb00ba,2,1710,2,12.0,3.0,0.5,"(2410, 33, 0.5)"
3,fb2c794175304c4caeb55e654270421f,a32736b04c27437da3078374d47af1b1,0149c6d63e214040b44d4a3789bb00ba,2018-04-13 18:18:05,<p>Hi Elisabeth! </p><p><br></p><p>If you are ...,72,0149c6d63e214040b44d4a3789bb00ba,34217a1861d640a58c85e033414cf9cb,2018-04-12 17:13:45+00:00,What is the best way to prepare for studying e...,...,a32736b04c27437da3078374d47af1b1,computer-software,0149c6d63e214040b44d4a3789bb00ba,2,1,2,1.0,3.0,0.5,"(18373, 33, 0.5)"
4,f3fc23809cda472780fc565334f35000,36ff3b3666df400f956f8335cf53e09e,acc611cfb5c44daa8a3d7d65dfffa5ff,2018-08-14 10:37:01,<p>The most important thing that you can do is...,102,acc611cfb5c44daa8a3d7d65dfffa5ff,5b751a8ee4a047f7a08ce9eb5e43e5a2,2018-08-14 04:49:33+00:00,How should I prepare myself for my job search ...,...,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",acc611cfb5c44daa8a3d7d65dfffa5ff,1,1710,1,12.0,4.0,1.0,"(2410, 47, 1.0)"


In [36]:
#df_merge['questions_date_added'] = pd.to_datetime(df_merge['questions_date_added']).dt.strftime('%Y-%m-%d %H:%M:%S')
#df_merge['students_date_joined'] = pd.to_datetime(df_merge['students_date_joined']).dt.strftime('%Y-%m-%d %H:%M:%S')

In [37]:
df_merge.head()

,answers_id,answers_author_id,answers_question_id,answers_date_added,answers_body,answers_id_num,questions_id,questions_author_id,questions_date_added,questions_title,...,tag_users_user_id,professionals_tag_name,id_y,score_y,num_of_ans_by_professional,num_ans_per_ques,num_tags_professional,num_tags_question,total_weights,author_question_id_tuple
0,4e5f01128cae4f6d8fd697cec5dca60c,36ff3b3666df400f956f8335cf53e09e,332a511f1569444485cf7a7a556a5e54,2016-04-29 19:40:14,<p>Hi!</p>\n<p>You are asking a very interesti...,0,332a511f1569444485cf7a7a556a5e54,8f6f374ffd834d258ab69d376dd998f5,2016-04-26 11:14:26+00:00,Teacher career question,...,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",332a511f1569444485cf7a7a556a5e54,1,1710,1,12.0,3.0,1.0,"(2410, 0, 1.0)"
1,f3519ab99a1a4a13a8a9ecb814287d2a,36ff3b3666df400f956f8335cf53e09e,0f1d6a4f276c4a05878dd48e03e52289,2016-07-31 15:35:54,<p>Hi Rodrigo!</p>\n<p>The important thing to ...,11,0f1d6a4f276c4a05878dd48e03e52289,585ac233015447cc9e9a217044e515e1,2016-05-19 22:16:25+00:00,what kind of college could i go to for a soc...,...,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",0f1d6a4f276c4a05878dd48e03e52289,1,1710,1,12.0,3.0,1.0,"(2410, 7, 1.0)"
2,825f6e316a5f48328d6f8af831df9940,36ff3b3666df400f956f8335cf53e09e,0149c6d63e214040b44d4a3789bb00ba,2018-04-15 23:08:46,<p>Congratulations on being interested in find...,71,0149c6d63e214040b44d4a3789bb00ba,34217a1861d640a58c85e033414cf9cb,2018-04-12 17:13:45+00:00,What is the best way to prepare for studying e...,...,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",0149c6d63e214040b44d4a3789bb00ba,2,1710,2,12.0,3.0,0.5,"(2410, 33, 0.5)"
3,fb2c794175304c4caeb55e654270421f,a32736b04c27437da3078374d47af1b1,0149c6d63e214040b44d4a3789bb00ba,2018-04-13 18:18:05,<p>Hi Elisabeth! </p><p><br></p><p>If you are ...,72,0149c6d63e214040b44d4a3789bb00ba,34217a1861d640a58c85e033414cf9cb,2018-04-12 17:13:45+00:00,What is the best way to prepare for studying e...,...,a32736b04c27437da3078374d47af1b1,computer-software,0149c6d63e214040b44d4a3789bb00ba,2,1,2,1.0,3.0,0.5,"(18373, 33, 0.5)"
4,f3fc23809cda472780fc565334f35000,36ff3b3666df400f956f8335cf53e09e,acc611cfb5c44daa8a3d7d65dfffa5ff,2018-08-14 10:37:01,<p>The most important thing that you can do is...,102,acc611cfb5c44daa8a3d7d65dfffa5ff,5b751a8ee4a047f7a08ce9eb5e43e5a2,2018-08-14 04:49:33+00:00,How should I prepare myself for my job search ...,...,36ff3b3666df400f956f8335cf53e09e,"engineering,computer-science,science,college,e...",acc611cfb5c44daa8a3d7d65dfffa5ff,1,1710,1,12.0,4.0,1.0,"(2410, 47, 1.0)"


**Building Data for LightFM Class**: From step 2 we already know that lightfm library except data in a very specific and elligent way. LightFM data format is already discussed in step 2. Feel free to read that. Now we are building a class that will be put all of dataset building puzzle in a specific class.

In [38]:
class LightFMDataPrep:
    def __init__(self):
        pass
    def create_features(self, dataframe, features_name, id_col_name):
        """
        Generate features that will be ready for feeding into lightfm

        Parameters
        ----------
        dataframe: Dataframe
            Pandas Dataframe which contains features
        features_name : List
            List of feature columns name avaiable in dataframe
        id_col_name: String
            Column name which contains id of the question or
            answer that the features will map to.
            There are two possible values for this variable.
            1. questions_id_num
            2. professionals_id_num

        Returns
        -------
        Pandas Series
            A pandas series containing process features
            that are ready for feed into lightfm.
            The format of each value
            will be (user_id, ['feature_1', 'feature_2', 'feature_3'])
            Ex. -> (1, ['military', 'army', '5'])
        """

        features = dataframe[features_name].apply(
            lambda x: ','.join(x.map(str)), axis=1)
        features = features.str.split(',')
        features = list(zip(dataframe[id_col_name], features))
        return features

    def generate_feature_list(self, dataframe, features_name):
        """
        Generate features list for mapping 

        Parameters
        ----------
        dataframe: Dataframe
            Pandas Dataframe for Users or Q&A. 
        features_name : List
            List of feature columns name avaiable in dataframe. 

        Returns
        -------
        List of all features for mapping 
        """
        features = dataframe[features_name].apply(
            lambda x: ','.join(x.map(str)), axis=1)
        features = features.str.split(',')
        features = features.apply(pd.Series).stack().reset_index(drop=True)
        return features
    
    def create_data(self, questions, professionals, merge):
        question_feature_list = self.generate_feature_list(
            questions,
            ['questions_tag_name'])

        professional_feature_list = self.generate_feature_list(
            professionals,
            ['professional_all_tags'])
        
        merge['total_weights'] = 1 / (
            merge['num_ans_per_ques'])
        
        # creating features for feeding into lightfm 
        questions['question_features'] = self.create_features(
            questions, ['questions_tag_name'], 
            'questions_id_num')

        professionals['professional_features'] = self.create_features(
            professionals,
            ['professional_all_tags'],
            'professionals_id_num')
        
        return question_feature_list,\
    professional_feature_list,merge,questions,professionals
        
    def fit(self, questions, professionals, merge):
        ########################
        # Dataset building for lightfm
        ########################
        question_feature_list, \
        professional_feature_list,\
        merge,questions,professionals = \
        self.create_data(questions, professionals, merge)
        
        # define our dataset variable
        # then we feed unique professionals and questions ids
        # and item and professional feature list
        # this will create lightfm internel mapping
        dataset = Dataset()
        dataset.fit(
            set(professionals['professionals_id_num']), 
            set(questions['questions_id_num']),
            item_features=question_feature_list, 
            user_features=professional_feature_list)

        # now we are building interactions
        # matrix between professionals and quesitons
        # we are passing professional and questions id as a tuple
        # e.g -> pd.Series((pro_id, question_id), (pro_id, questin_id))
        # then we use lightfm build in method for building interactions matrix
        merge['author_question_id_tuple'] = list(zip(
            merge.professionals_id_num,
            merge.questions_id_num,
            merge.total_weights))

        interactions, weights = dataset.build_interactions(
            merge['author_question_id_tuple'])

        # now we are building our questions and
        # professionals features
        # in a way that lightfm understand.
        # we are using lightfm build in method for building
        # questions and professionals features 
        questions_features = dataset.build_item_features(
            questions['question_features'])

        professional_features = dataset.build_user_features(
            professionals['professional_features'])
        
        return interactions, weights, questions_features, professional_features

**Train Model Class**: In step 2, we saw how we build and train our model. Now we are going to put those all together in TrainLightFM class.

In [39]:
class TrainLightFM:
    def __init__(self):
        pass
        
    def train_test_split(self, interactions, weights):
        train_interactions, test_interactions = \
        cross_validation.random_train_test_split(
            interactions, 
            random_state=np.random.RandomState(2019))
        
        train_weights, test_weights = \
        cross_validation.random_train_test_split(
            weights, 
            random_state=np.random.RandomState(2019))
        return train_interactions,\
    test_interactions, train_weights, test_weights
    
    def fit(self, interactions, weights,
            questions_features, professional_features,
            cross_validation=False,no_components=150,
            learning_rate=0.05,
            loss='warp',
            random_state=2019,
            verbose=True,
            num_threads=4, epochs=5):

        # Model building part
        # define lightfm model by specifying hyper-parametre
        # then fit the model with ineteractions matrix,
        # item and user features
        
        model = LightFM(
            no_components,
            learning_rate,
            loss=loss,
            random_state=random_state)
        model.fit(
            interactions,
            item_features=questions_features,
            user_features=professional_features, sample_weight=weights,
            epochs=epochs, num_threads=num_threads, verbose=verbose)
        
        return model

**Recommendations classs**: Now we are going to build a class for making recommendations. This will make easy for making recommendations in djono api. This recommendations class build with extra features. You can use this for general prediction by giving professionals ids and questions features. It has another features that let's choose questions from range of two dates and make recommendation from those questions. 

This is useful because those professionals that choose email frequency lavel as "weekly" or "daily", we can select questions from a week and then recommend those questions. 

In [40]:
class LightFMRecommendations:
    """
    Make prediction given model and professional ids
    """
    def __init__(self, lightfm_model,
                 professionals_features,
                 questions_features,
                 questions,
                 professionals,merge):
        self.model = lightfm_model
        self.professionals_features = professionals_features
        self.questions_features = questions_features
        self.questions = questions
        self.professionals = professionals
        self.merge = merge
        
    def previous_answered_questions(self, professionals_id):
        previous_q_id_num = (
            self.merge.loc[\
                self.merge['professionals_id_num'] == \
                professionals_id]['questions_id_num'])
        
        previous_answered_questions = self.questions.loc[\
            self.questions['questions_id_num'].isin(
            previous_q_id_num)]
        
        return previous_answered_questions
        
    def _filter_question_by_pro(self, professionals_id):
        """Drop questions that professional already answer"""
        previous_answered_questions = \
        self.previous_answered_questions(professionals_id)
        
        discard_qu_id = \
        previous_answered_questions['questions_id_num'].values.tolist()
        
        questions_for_prediction = \
        self.questions.loc[~self.questions['questions_id_num'].isin(discard_qu_id)]
        
        return questions_for_prediction
    
    def _filter_question_by_date(self, questions, start_date, end_date):
        mask = \
        (questions['questions_date_added'] > start_date) & \
        (questions['questions_date_added'] <= end_date)
        
        return questions.loc[mask]
        
    def recommend_by_pro_id_general(self,
                                    professional_id,
                                    num_prediction=8):
        questions_for_prediction = self._filter_question_by_pro(professional_id)
        score = self.model.predict(
            professional_id,
            questions_for_prediction['questions_id_num'].values.tolist(), 
            item_features=self.questions_features,
            user_features=self.professionals_features)
        
        questions_for_prediction['recommendation_score'] = score
        questions_for_prediction = questions_for_prediction.sort_values(
            by='recommendation_score', ascending=False)[:num_prediction]
        
        return questions_for_prediction
    
    def recommend_by_pro_id_frequency_date_range(self,
                                                 professional_id,
                                                 start_date,
                                                 end_date,
                                                 num_prediction=8):
        questions_for_prediction = \
        self._filter_question_by_pro(professional_id)
        
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        
        questions_for_prediction = self._filter_question_by_date(
            questions_for_prediction, start_date, end_date)
        
        score = self.model.predict(
            professional_id,
            questions_for_prediction['questions_id_num'].values.tolist(), 
            item_features=self.questions_features,
            user_features=self.professionals_features)
        
        questions_for_prediction['recommendation_score'] = score
        questions_for_prediction = questions_for_prediction.sort_values(
            by='recommendation_score', ascending=False)[:num_prediction]
        return questions_for_prediction

**Modified Class**

In [41]:
class LightFMRecommendations:
    """
    Make prediction given model and professional ids
    """
    def __init__(self, lightfm_model,
                 professionals_features,
                 questions_features,
                 questions,
                 professionals, merge):
        self.model = lightfm_model
        self.professionals_features = professionals_features
        self.questions_features = questions_features
        self.questions = questions.copy()
        self.questions['questions_date_added'] = pd.to_datetime(self.questions['questions_date_added'])
        self.professionals = professionals
        self.merge = merge
        
    def previous_answered_questions(self, professionals_id):
        previous_q_id_num = (
            self.merge.loc[\
                self.merge['professionals_id_num'] == \
                professionals_id]['questions_id_num'])
        
        previous_answered_questions = self.questions.loc[\
            self.questions['questions_id_num'].isin(
            previous_q_id_num)]
        
        return previous_answered_questions
        
    def _filter_question_by_pro(self, professionals_id):
        """Drop questions that professional already answer"""
        previous_answered_questions = \
        self.previous_answered_questions(professionals_id)
        
        discard_qu_id = \
        previous_answered_questions['questions_id_num'].values.tolist()
        
        questions_for_prediction = \
        self.questions.loc[~self.questions['questions_id_num'].isin(discard_qu_id)]
        
        return questions_for_prediction
    
    def _filter_question_by_date(self, questions, start_date, end_date):
        mask = \
        (questions['questions_date_added'] > start_date) & \
        (questions['questions_date_added'] <= end_date)
        
        return questions.loc[mask]
        
    def recommend_by_pro_id_general(self,
                                    professional_id,
                                    num_prediction=8):
        questions_for_prediction = self._filter_question_by_pro(professional_id)
        score = self.model.predict(
            professional_id,
            questions_for_prediction['questions_id_num'].values.tolist(), 
            item_features=self.questions_features,
            user_features=self.professionals_features)
        
        questions_for_prediction['recommendation_score'] = score
        questions_for_prediction = questions_for_prediction.sort_values(
            by='recommendation_score', ascending=False)[:num_prediction]
        
        return questions_for_prediction
    
    def recommend_by_pro_id_frequency_date_range(self,
                                                 professional_id,
                                                 start_date,
                                                 end_date,
                                                 num_prediction=8):
        questions_for_prediction = \
        self._filter_question_by_pro(professional_id)
        
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        
        questions_for_prediction = self._filter_question_by_date(
            questions_for_prediction, start_date, end_date)
        
        score = self.model.predict(
            professional_id,
            questions_for_prediction['questions_id_num'].values.tolist(), 
            item_features=self.questions_features,
            user_features=self.professionals_features)
        
        questions_for_prediction['recommendation_score'] = score
        questions_for_prediction = questions_for_prediction.sort_values(
            by='recommendation_score', ascending=False)[:num_prediction]
        return questions_for_prediction

**Put it all together**: Now we defined all our important class file. Let's use each of these class and build our model.

In [42]:
# instiate all class instance
cv_data_prep = CareerVillageDataPreparation()
light_fm_data_prep = LightFMDataPrep()
train_lightfm = TrainLightFM()

# process raw data
df_questions_p, df_professionals_p, df_merge_p = \
cv_data_prep.prepare(
    df_professionals,df_students,
    df_questions,df_answers,
    df_tags,df_tag_questions,df_tag_users,
    df_question_scores)

# Additional lines
# df_questions_p #questions_date_added #students_date_joined
df_questions_p['questions_date_added'] = pd.to_datetime(df_questions_p['questions_date_added']).dt.strftime('%Y-%m-%d %H:%M:%S')
df_questions_p['students_date_joined'] = pd.to_datetime(df_questions_p['students_date_joined']).dt.strftime('%Y-%m-%d %H:%M:%S')
#df_professionals_p #professionals_date_joined
df_professionals_p['professionals_date_joined'] = pd.to_datetime(df_professionals_p['professionals_date_joined']).dt.strftime('%Y-%m-%d %H:%M:%S')
#df_merge_p #answers_date_added #questions_date_added
df_merge_p['answers_date_added'] = pd.to_datetime(df_merge_p['answers_date_added']).dt.strftime('%Y-%m-%d %H:%M:%S')
df_merge_p['questions_date_added'] = pd.to_datetime(df_merge_p['questions_date_added']).dt.strftime('%Y-%m-%d %H:%M:%S')

# prepare data for lightfm 
interactions, weights, \
questions_features, professional_features = \
light_fm_data_prep.fit(
    df_questions_p, df_professionals_p, df_merge_p)

# finally build and trian our model
model = train_lightfm.fit(interactions,
                          weights,
                          questions_features,
                          professional_features)

Epoch: 100%|██████████| 5/5 [00:17<00:00,  3.50s/it]


Awesome! Do you see, how easy it was for building our model. We can surely apply this idea when putting the model into production. Now we are going to see some real recommendations.

In [43]:
# define our recommender class
lightfm_recommendations = LightFMRecommendations(
    model,
    professional_features,questions_features,
    df_questions_p, df_professionals_p, df_merge_p)

# let's what our model predict for user id 3
print("Recommendation for professional: " + str(3))
display(lightfm_recommendations.recommend_by_pro_id_general(3)[:8])

Recommendation for professional: 3


,questions_id,questions_author_id,questions_date_added,questions_title,questions_body,questions_id_num,tag_questions_question_id,questions_tag_name,id,score,students_id,students_location,students_date_joined,students_id_num,question_features,recommendation_score
2423,9515b833b2ac4092a8b1a8cdb380781f,941ae126a59745fa9b4556293b38c1fb,2019-01-08 20:47:44,How long does it take to become a Detective?,#law #criminal-justice #lawyer #police #law-en...,2423,9515b833b2ac4092a8b1a8cdb380781f,"police,law,law-enforcement,criminal-justice,la...",9515b833b2ac4092a8b1a8cdb380781f,2,941ae126a59745fa9b4556293b38c1fb,"Oakland, California",2019-01-08 20:35:58,30755.0,"(2423, [police, law, law-enforcement, criminal...",-2.365361
17184,570ca25a625d461abffac230ea110db5,941ae126a59745fa9b4556293b38c1fb,2019-01-10 01:48:47,What types of Detectives are there?,#law #criminal-justice #lawyer #law-enforcemen...,17184,570ca25a625d461abffac230ea110db5,"police,law,law-enforcement,criminal-justice,la...",570ca25a625d461abffac230ea110db5,2,941ae126a59745fa9b4556293b38c1fb,"Oakland, California",2019-01-08 20:35:58,30755.0,"(17184, [police, law, law-enforcement, crimina...",-2.387972
9778,776e22d9eb1045eb8a9771eb015e8ddf,d7601a6cc1d04e61aaa16c95cbd0b128,2018-10-03 14:04:13,I want to be a police officer or a police disp...,#police-officer #law #law-enforcement #crimina...,9778,776e22d9eb1045eb8a9771eb015e8ddf,"police,police-officer,law,law-enforcement,crim...",776e22d9eb1045eb8a9771eb015e8ddf,2,d7601a6cc1d04e61aaa16c95cbd0b128,"Olney, Illinois",2018-10-03 14:01:25,29951.0,"(9778, [police, police-officer, law, law-enfor...",-2.730303
16214,ccb15b06a96a4bcfb4d5844550af25cc,8a8305d32bd144d5877842dcabdfb6d7,2016-05-04 16:32:58,"Do you go to college, then B.L.E.T( Basic Law ...",I am an explorer and is trying to set my caree...,16214,ccb15b06a96a4bcfb4d5844550af25cc,"police,law,law-enforcement",ccb15b06a96a4bcfb4d5844550af25cc,2,8a8305d32bd144d5877842dcabdfb6d7,"Laurinburg, North Carolina",2016-05-02 16:37:52,7103.0,"(16214, [police, law, law-enforcement])",-2.759122
18126,6c0079d59ae74c1388045fecbe585570,8a8305d32bd144d5877842dcabdfb6d7,2016-05-04 16:34:56,Do you need law enforcement background such as...,I am an explorer and is trying to set my caree...,18126,6c0079d59ae74c1388045fecbe585570,"police,law,law-enforcement",6c0079d59ae74c1388045fecbe585570,4,8a8305d32bd144d5877842dcabdfb6d7,"Laurinburg, North Carolina",2016-05-02 16:37:52,7103.0,"(18126, [police, law, law-enforcement])",-2.762117
18872,c3e6e57cb27b4134be9b8608a711e2fc,43f813594dd44e16843ecae4e2362ead,2015-03-23 21:17:34,What majors would fit a law enforcement career?,Im asking this question because I've heard tha...,18872,c3e6e57cb27b4134be9b8608a711e2fc,"police,law,law-enforcement",c3e6e57cb27b4134be9b8608a711e2fc,4,43f813594dd44e16843ecae4e2362ead,"Los Angeles, California",2015-03-23 21:09:01,3322.0,"(18872, [police, law, law-enforcement])",-2.796726
10534,322afc67ac1848188a4a6d2bf5c51b20,8a8305d32bd144d5877842dcabdfb6d7,2016-05-05 15:42:36,Is there any required college courses to becom...,I am an explorer and is trying to set my caree...,10534,322afc67ac1848188a4a6d2bf5c51b20,"police,law,law-enforcement",322afc67ac1848188a4a6d2bf5c51b20,2,8a8305d32bd144d5877842dcabdfb6d7,"Laurinburg, North Carolina",2016-05-02 16:37:52,7103.0,"(10534, [police, law, law-enforcement])",-2.803918
18947,cdd0b274ec8f4122a39989707342ccfe,8a8305d32bd144d5877842dcabdfb6d7,2016-05-05 15:39:53,"Could I go straight into Law Enforcememt, when...",I am an explorer and is trying to set my caree...,18947,cdd0b274ec8f4122a39989707342ccfe,"police,law,law-enforcement",cdd0b274ec8f4122a39989707342ccfe,4,8a8305d32bd144d5877842dcabdfb6d7,"Laurinburg, North Carolina",2016-05-02 16:37:52,7103.0,"(18947, [police, law, law-enforcement])",-2.833616


In [44]:
# also let's see what our model predicts for professional 3
# given questions between two dates
print("Recommendations for professionals (question from 2016-1-1 to 2016-12-31): " + str(3))
display(lightfm_recommendations.recommend_by_pro_id_frequency_date_range(3,'2016-1-1','2016-12-31')[:8])

Recommendations for professionals (question from 2016-1-1 to 2016-12-31): 3


,questions_id,questions_author_id,questions_date_added,questions_title,questions_body,questions_id_num,tag_questions_question_id,questions_tag_name,id,score,students_id,students_location,students_date_joined,students_id_num,question_features,recommendation_score
16214,ccb15b06a96a4bcfb4d5844550af25cc,8a8305d32bd144d5877842dcabdfb6d7,2016-05-04 16:32:58,"Do you go to college, then B.L.E.T( Basic Law ...",I am an explorer and is trying to set my caree...,16214,ccb15b06a96a4bcfb4d5844550af25cc,"police,law,law-enforcement",ccb15b06a96a4bcfb4d5844550af25cc,2,8a8305d32bd144d5877842dcabdfb6d7,"Laurinburg, North Carolina",2016-05-02 16:37:52,7103.0,"(16214, [police, law, law-enforcement])",-2.759122
18126,6c0079d59ae74c1388045fecbe585570,8a8305d32bd144d5877842dcabdfb6d7,2016-05-04 16:34:56,Do you need law enforcement background such as...,I am an explorer and is trying to set my caree...,18126,6c0079d59ae74c1388045fecbe585570,"police,law,law-enforcement",6c0079d59ae74c1388045fecbe585570,4,8a8305d32bd144d5877842dcabdfb6d7,"Laurinburg, North Carolina",2016-05-02 16:37:52,7103.0,"(18126, [police, law, law-enforcement])",-2.762117
10534,322afc67ac1848188a4a6d2bf5c51b20,8a8305d32bd144d5877842dcabdfb6d7,2016-05-05 15:42:36,Is there any required college courses to becom...,I am an explorer and is trying to set my caree...,10534,322afc67ac1848188a4a6d2bf5c51b20,"police,law,law-enforcement",322afc67ac1848188a4a6d2bf5c51b20,2,8a8305d32bd144d5877842dcabdfb6d7,"Laurinburg, North Carolina",2016-05-02 16:37:52,7103.0,"(10534, [police, law, law-enforcement])",-2.803918
18947,cdd0b274ec8f4122a39989707342ccfe,8a8305d32bd144d5877842dcabdfb6d7,2016-05-05 15:39:53,"Could I go straight into Law Enforcememt, when...",I am an explorer and is trying to set my caree...,18947,cdd0b274ec8f4122a39989707342ccfe,"police,law,law-enforcement",cdd0b274ec8f4122a39989707342ccfe,4,8a8305d32bd144d5877842dcabdfb6d7,"Laurinburg, North Carolina",2016-05-02 16:37:52,7103.0,"(18947, [police, law, law-enforcement])",-2.833616
1941,716e1eb45ae64de29633eacf2ebddc0e,0a49a80de472412988aac14f93b06374,2016-07-22 03:52:32,What are important characteristics of a lawyer?,I was curious about the desirable traits of a ...,1941,716e1eb45ae64de29633eacf2ebddc0e,"law-enforcement,law-school,law,lawyer",716e1eb45ae64de29633eacf2ebddc0e,3,0a49a80de472412988aac14f93b06374,"Plano, Texas",2016-05-30 21:08:55,11780.0,"(1941, [law-enforcement, law-school, law, lawy...",-2.921825
8863,9cc2279d1e8e4a1780426bf93d7468ab,779a47c6bd16423992b81dc7b494e763,2016-07-11 00:47:33,What qualifications are needed to be promoted ...,What qualifications are needed to be promoted ...,8863,9cc2279d1e8e4a1780426bf93d7468ab,"criminal-justice,police,law-enforcement",9cc2279d1e8e4a1780426bf93d7468ab,6,779a47c6bd16423992b81dc7b494e763,NaN,2016-07-05 22:23:12,13386.0,"(8863, [criminal-justice, police, law-enforcem...",-3.005390
5139,33da5b42af304513af00f210be56ea68,779a47c6bd16423992b81dc7b494e763,2016-07-05 22:45:34,What do you wish you had known before you went...,I am currently in college and hope to pursue a...,5139,33da5b42af304513af00f210be56ea68,"police,law,law-enforcement,criminal-investigat...",33da5b42af304513af00f210be56ea68,4,779a47c6bd16423992b81dc7b494e763,NaN,2016-07-05 22:23:12,13386.0,"(5139, [police, law, law-enforcement, criminal...",-3.074200
22036,c3ea3de927884725a4ddd934fa691422,4ea5755ee38648a684354938b9829b55,2016-05-05 13:33:27,What type of training is required to become a ...,I want to know the type of training needed. #c...,22036,c3ea3de927884725a4ddd934fa691422,"criminal-justice,law-enforcement",c3ea3de927884725a4ddd934fa691422,2,4ea5755ee38648a684354938b9829b55,"Laurinburg, North Carolina",2016-05-02 14:05:33,7076.0,"(22036, [criminal-justice, law-enforcement])",-3.088016


Awesome! We can see our recommendations. Also, we can see, the new recommendation class has a method for recommending questions by a frequency of date. This is very helpful for recommending questions to professionals that have set their email frequency to daily or weekly. 

# Conclusion

**Idea that I tried but don't implemented in this notebook**: 
* Adding location features: I tried adding location features but somehow it decreases model AUC score to 91% to 84%. That's why I don't use that features.
* Adding dates and hearts data: I also tried that but it doesn't improve AUC score. 
* Correcting spelling error: I tried this method and successfully implemented it. But this is really slow. For that reason, I exluded it. 

**Idea that I think is important don't implemented in this notebook**: 
* Adding professionals industry and title as a features. This will inhance our model diversity and will increase overall recommendations score.
* CareerVillage should auto correct the hashtags for students questions asking time. This will help the model to match the tags more efficiently. 
* For those professionals those have choosen email frequency to immediete, we can create another same model just exchange user/item features. I mean train our model by giving questions as users and professionals as items. In this way, we can predict professionals by giving a questions. So that it helps to target daily frequency professionals.

Finally we came to end. I want give you a big thank you for reading this notebook. I have provided a very good recommender system for CareerVillage in the notebook. If you find any mistakes or have any suggestions feel free to comment. And don't forget to upvote. Good luck!

References: 

[1] [Improving Pairwise Learning for Item Recommendation from Implicit Feedback](http://webia.lip6.fr/~gallinar/gallinari/uploads/Teaching/WSDM2014-rendle.pdf)

[2] [Content-based filtering](https://towardsdatascience.com/what-are-product-recommendation-engines-and-the-various-versions-of-them-9dcab4ee26d5)

[3] [What Is Content-Based Filtering?](https://www.upwork.com/hiring/data/what-is-content-based-filtering/)

[4] [What Is Collaborative Filtering?](https://www.upwork.com/hiring/data/how-collaborative-filtering-works/)

[5] [Collaborative filtering](https://en.wikipedia.org/wiki/Collaborative_filtering)

[6] [LightFM model documentation](https://lyst.github.io/lightfm/docs/lightfm.html)

[7] [Metadata Embeddings for User and Item Cold-start Recommendations](https://arxiv.org/pdf/1507.08439.pdf)